In [21]:
import os

os.chdir(r'D:\Desktop\MachineLearning\NLP\NLP新闻分类赛')

In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse as sp

# 读取数据集
train_df = pd.read_csv('./data/train_set.csv', sep='\t')
test_df = pd.read_csv('./data/test_a.csv', sep='\t')
# 初始化TF-IDF向量化器
tfidf = TfidfVectorizer(
    analyzer='word',  # 特征提取基于单词
    ngram_range=(1, 3),  # 提取一元组和二元组作为特征
    min_df=3,  # 忽略在少于 3 个文档中出现的词
    max_df=0.9,  # 它将忽略那些在超过 90% 的文档中出现的词
    use_idf=True,  # 使用逆文档频率对词频进行加权
    max_features=3000,  # 不限制特征的数量
    smooth_idf=True,  # 在 IDF 计算中添加 1 进行平滑处理
    sublinear_tf=True  # 对词频进行子线性缩放，使用 1 + log(TF) 代替 TF
)

# 将文本转化为TF-IDF特征
train_tfidf = tfidf.fit_transform(train_df['text'])
test_tfidf = tfidf.transform(test_df['text'])

# 打印矩阵类型
print(type(train_tfidf))
# 将稀疏矩阵保存为 .npz 格式
sp.save_npz('./preprocess/train_tfidf.npz', train_tfidf)
sp.save_npz('./preprocess/test_tfidf.npz', test_tfidf)

<class 'scipy.sparse._csr.csr_matrix'>


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score

# 3. 分割数据集为训练集和测试集（80%训练集，20%测试集）
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. 初始化随机森林分类器
rf = RandomForestClassifier(random_state=42)

# 5. 设置超参数搜索空间
param_grid = {
    'max_depth': [None],
    'n_estimators': range(100, 301, 100),  # 决策树的数量
    'min_samples_split': [10, 20, 30],  # 分割内部节点所需的最小样本数
    'min_samples_leaf': [10, 20, 30],  # 叶子节点的最小样本数
    'max_features': ['sqrt', 'log2', None]  # 在每个决策树分裂时选择的最大特征数
}

# 6. 使用 HalvingGridSearchCV 进行超参数调优
halving_grid_search = HalvingGridSearchCV(estimator=rf, param_grid=param_grid,
                                          factor=3,  # 每次折半搜索空间
                                          max_resources=50000,  # 自动选择最大资源
                                          cv=2,  # k折交叉验证
                                          verbose=2,
                                          n_jobs=-1,
                                          scoring=make_scorer(f1_score, average='macro')  # 设置 F1 分数作为评分标准
                                          )

# 7. 训练模型
halving_grid_search.fit(X, y)

# 8. 输出最优超参数
print("Best parameters found: ", halving_grid_search.best_params_)
print("Best score: ", -halving_grid_search.best_score_)

# 9. 使用最优模型进行预测
best_rf = halving_grid_search.best_estimator_

In [ ]:
# 获取模型的所有参数
all_params = best_rf.get_params()
print(all_params)

In [ ]:
train_df = pd.read_csv('data/train_set.csv', sep='\t')
test_df = pd.read_csv('data/test_a.csv', sep='\t')
X_train = sp.load_npz('preprocess/train_tfidf.npz')
y_train = train_df['label'].values
X_test = sp.load_npz('preprocess/test_tfidf.npz')
print(type(y_train))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np
import joblib

# 初始化随机森林分类器
rf = RandomForestClassifier(random_state=42,
                            n_estimators=1000,
                            max_depth=None,
                            max_features='sqrt',
                            min_samples_split=20,
                            min_samples_leaf=20,
                            n_jobs=-1,
                            verbose=2)

# 初始化 StratifiedKFold 进行分层五折交叉验证
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 存储每折的验证集和测试集的预测结果
val_predictions = []
test_predictions = []
# 存储每折的验证集和测试集的真实标签
val_true_labels = []
test_true_labels = []

# 进行五折交叉验证
for train_index, val_index in skf.split(X_train, y_train):
    # 分割训练集和验证集
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # 训练模型
    rf.fit(X_train_fold, y_train_fold)

    # 在验证集上进行预测
    y_val_pred = rf.predict(X_val_fold)
    val_predictions.append(y_val_pred)
    val_true_labels.append(y_val_fold)

    # 在测试集上进行预测
    y_test_pred = rf.predict(X_test)
    test_predictions.append(y_test_pred)

# 你可以将预测结果和真实标签转换为 numpy 数组，方便后续处理
val_predictions = np.array(val_predictions)
test_predictions = np.array(test_predictions)
val_true_labels = np.array(val_true_labels)

# 计算验证集和测试集的评估指标（例如 F1 分数）
from sklearn.metrics import f1_score

# 计算每折验证集的 F1 分数
val_f1_scores = []
for i in range(5):
    f1 = f1_score(val_true_labels[i], val_predictions[i], average='macro')
    val_f1_scores.append(f1)
print("F1 scores on validation sets:", val_f1_scores)


In [23]:
# 整合测试集预测结果
# 假设我们取多数投票结果作为最终的测试集预测结果

y_test_pred = rf.predict(test_tfidf)
# final_test_pred = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=test_predictions)
test_df['label'] = y_test_pred
test_df['label'].to_csv("./result/submission_randomforest.csv", index=False)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    0.9s
[Parallel(n_jobs=16)]: Done 333 tasks      | elapsed:    2.2s
[Parallel(n_jobs=16)]: Done 616 tasks      | elapsed:    3.9s
[Parallel(n_jobs=16)]: Done 1000 out of 1000 | elapsed:    6.3s finished


In [ ]:
import joblib

# 保存模型
# joblib.dump(final_rf, './model/rf_clf.joblib')
# 加载模型
# best_rf_loaded = joblib.load('rf_clf.joblib')

## xgboost

In [ ]:
import xgboost as xgb
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import make_scorer, f1_score

X = train_test
y = train_df['label']  # 假设标签列名为 'label'

# 4. 初始化 XGBoost 分类器
xgb_clf = xgb.XGBClassifier(
    random_state=42,  # 设置随机种子，确保结果可重复
    booster='gbtree',  # 使用梯度提升树作为提升类型
    learning_rate=0.1,  # 学习率，控制每棵树的贡献
    objective='multi:softmax',  # 目标函数设置为多分类任务，使用 softmax 作为多分类的目标函数
    subsample=0.8,  # 训练数据的子样本比例（随机选择样本，防止过拟合）
    colsample_bytree=0.5,  # 每棵树随机选择特征的比例
    reg_lambda=10,  # L2 正则化项的权重
    nthread=-1,  # 使用的 CPU 核心数，-1 表示使用所有可用核心
    num_class=19,  # 类别数量（多分类任务的类别数）
    # XGBoost 中没有 colsample_bylevel 参数，可以使用 colsample_bynode 代替，不过功能不完全相同
    # min_child_weight=1.5,       # XGBoost 中对应的是 min_child_weight，但需要注意它们的计算方式可能有细微差异
    eval_metric='mlogloss'  # 评估指标使用多分类对数损失
)

# 5. 设置超参数搜索空间（适用于XGBoost）
param_grid = {
    'n_estimators': range(100, 301, 100),  # 决策树的数量
    'max_depth': range(10, 21, 5),  # 树的最大深度
    'colsample_bytree': [0.8, 1.0],  # 每棵树的列采样比例
    'min_child_weight': [1, 3, 5]  # 每个叶子节点的最小样本权重
}

# 6. 使用 HalvingGridSearchCV 进行超参数调优
halving_grid_search = HalvingGridSearchCV(estimator=xgb_clf, param_grid=param_grid,
                                          factor=3,  # 每次折半搜索空间
                                          max_resources=50000,  # 自动选择最大资源
                                          cv=3,  # 2折交叉验证
                                          verbose=1,
                                          n_jobs=-1,
                                          scoring=make_scorer(f1_score, average='macro')  # 设置 F1 分数作为评分标准
                                          )

# 7. 训练模型
halving_grid_search.fit(X, y)

# 8. 输出最优超参数
print("Best parameters found: ", halving_grid_search.best_params_)
print("Best score: ", -halving_grid_search.best_score_)

# 9. 使用最优模型进行预测
best_xgb = halving_grid_search.best_estimator_


In [ ]:
import lightgbm as lgb
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import make_scorer, f1_score

# 假设 train_test 和 train_df 数据已准备好
X = train_test
y = train_df['label']  # 假设标签列名为 'label'

# 4. 初始化 LightGBM 分类器
lgbm = lgb.LGBMClassifier(random_state=42,  # 设置随机种子，确保结果可重复
                          boosting_type='gbdt',  # 使用梯度提升树（Gradient Boosting Decision Tree）作为提升类型
                          max_depth=-1,  # 树的最大深度，-1表示没有限制
                          learning_rate=0.1,  # 学习率，控制每棵树的贡献
                          objective='multiclass',  # 目标函数设置为多分类任务
                          subsample=0.7,  # 训练数据的子样本比例（随机选择样本，防止过拟合）
                          colsample_bytree=0.5,  # 每棵树随机选择特征的比例
                          reg_lambda=10,  # L2 正则化项的权重
                          n_jobs=-1,  # 使用的CPU核心数，-1表示使用所有可用核心
                          num_class=19,  # 类别数量（多分类任务的类别数）
                          colsample_bylevel=0.5,  # 每一层的特征采样比例
                          min_child_weight=1.5,  # 叶子节点最小的样本权重和，用于防止过拟合
                          metric='multi_logloss')  # 评估指标使用多分类对数损失

# 5. 设置超参数搜索空间
param_grid = {
    'n_estimators': range(10, 511, 100),  # 基学习器的数量
    'num_leaves': [31, 50, 80],  # 最大叶子数
}

# 6. 使用 HalvingGridSearchCV 进行超参数调优
halving_grid_search = HalvingGridSearchCV(estimator=lgbm, param_grid=param_grid,
                                          factor=3,  # 每次折半搜索空间
                                          max_resources=100000,  # 自动选择最大资源
                                          cv=3,  # 2折交叉验证
                                          verbose=2,
                                          n_jobs=-1,
                                          scoring=make_scorer(f1_score, average='macro')  # 设置 F1 分数作为评分标准
                                          )

# 7. 训练模型
halving_grid_search.fit(X, y)

# 8. 输出最优超参数
print("Best parameters found: ", halving_grid_search.best_params_)
print("Best score: ", -halving_grid_search.best_score_)

# 9. 使用最优模型进行预测
best_lgbm = halving_grid_search.best_estimator_
